## 1 - Importações

In [104]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, accuracy_score

In [3]:
df = pd.read_csv(
    'PETR4_textuais_numericos.csv',
    encoding='utf-8-sig', 
    index_col='date'
)

In [4]:
df.shape

(3622, 13)

In [6]:
df.columns

Index(['close', 'compound_gn', 'negative_gn', 'neutral_gn', 'positive_gn',
       'compound_tw', 'negative_tw', 'neutral_tw', 'positive_tw', 'open',
       'high', 'low', 'volume'],
      dtype='object')

## 2 - Funções auxiliares

### 2.1 Calcular os Indicadores

In [15]:
import pandas as pd

def calcular_mms(df, periodo):
    """
    Calcula a Média Móvel Simples (MMS).
    
    Args:
        precos (list ou pd.Series): Série de preços.
        periodo (int): Período da média móvel.
    
    Returns:
        pd.Series: Média Móvel Simples.
    """
    df[f'mms_{periodo}'] = df['close'].rolling(window=periodo).mean()
    
    return df

def calcular_mme(df, periodo):
    """
    Calcula a Média Móvel Exponencial (MME).
    
    Args:
        precos (list ou pd.Series): Série de preços.
        periodo (int): Período da média móvel.
    
    Returns:
        pd.Series: Média Móvel Exponencial.
    """
    df[f'mme_{periodo}'] = df['close'].ewm(span=periodo, adjust=False).mean()
    
    return df

def calcular_ifr(df, periodo=14):
    """
    Calcula o Índice de Força Relativa (IFR ou RSI - Relative Strength Index).
    
    Args:
        precos (list ou pd.Series): Série de preços.
        periodo (int): Período do IFR.
    
    Returns:
        pd.Series: Valores do IFR.
    """
    precos = df['close']
    variacao = precos.diff(1)
    
    ganho = variacao.where(variacao > 0, 0)
    perda = -variacao.where(variacao < 0, 0)
    
    media_ganho = ganho.rolling(window=periodo).mean()
    media_perda = perda.rolling(window=periodo).mean()
    
    rs = media_ganho / media_perda
    ifr = 100 - (100 / (1 + rs))
    df[f'ifr_{periodo}'] = ifr
    
    return df

### 2.2 Carregar Dados

In [74]:
# Exemplo de função para obter os dados das ações
def get_stock_data(stock):
    """
    Função para carregar os dados uma ação.

    Parâmetros:
        stock (str): Nome da ação.

    Retorna:
        tuple: DataFrame com os dados da ação e o nome da coluna de destino (target).
    """
    # Carregando
    df = pd.read_csv(
    f'{stock}_textuais_numericos.csv',
    encoding='utf-8-sig', 
    index_col='date'
    )

    # Calcular os indicadores técnicos
    df = calcular_mms(df, 30)
    df = calcular_mms(df, 5)
    df = calcular_mme(df, 5)
    df = calcular_mme(df, 30)
    df = calcular_ifr(df)

    # Limpar os dados
    df.dropna(inplace=True)

    return df

In [77]:
def load_stocks_data(stocks):
    """
    Carrega os dados das empresas usando a função get_stock_data e armazena-os em um dicionário.

    Parâmetros:
        stocks (list): Lista de códigos das ações.

    Retorna:
        dict: Dicionário com os códigos das ações como chave e os dados como valor.
    """
    stock_data_dict = {}
    
    for stock in stocks:
        try:
            print(f"Carregando dados para: {stock}")
            stock_data = get_stock_data(stock)  # Obtém os dados da função
            stock_data_dict[stock] = stock_data
        except Exception as e:
            print(f"Erro ao carregar dados para {stock}: {e}")
    
    return stock_data_dict

### 2.3 Combinações dos Dados

In [76]:
def make_combination_data(df, num_combination):
    """
    Combina dados com base em um identificador e retorna o DataFrame combinado e uma descrição.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados originais.
        num_combination (int): Número identificador da combinação.

    Retorna:
        tuple: DataFrame combinado e descrição da combinação.
    """
    # Definindo as combinações
    # Stock Data
    stock_data = ['open','close','high','low','volume']
    # Google News
    google_news = ['compound_gn', 'negative_gn', 'neutral_gn', 'positive_gn']
    # Twitter
    twitter = ['compound_tw', 'negative_tw', 'neutral_tw', 'positive_tw']
    # MMS
    mms = ['mms_30', 'mms_5']
    # MME
    mme = ['mme_5', 'mme_30']
    # IFR de 14 periodos
    ifr = ['ifr_14']
    # Valor de fechamento
    close = ['close']
    
    # Mapear combinações para descrição e colunas
    combinations = {
        1: ("Stock Data", stock_data),
        2: ("Stock Data + Google News", stock_data + google_news),
        3: ("Stock Data + Twitter", stock_data + twitter),
        4: ("Stock Data + IT (IFR + MMS + MME)", stock_data + ifr + mms + mme),
        5: ("Google News + Twitter + IFR + MMS", close + google_news + twitter + ifr + mms),
        6: ("Google News + Twitter + IFR + MME", close + google_news + twitter + ifr + mme),
        7: ("Google News + Twitter + IFR + MME + MMS", close + google_news + twitter + ifr + mme + mms),
    }

    # Verifica se a combinação existe
    if num_combination not in combinations:
        raise ValueError(f"Combinação {num_combination} não é válida. Escolha entre 1 e {len(combinations)}.")

    # Recupera descrição e colunas da combinação
    description_combination, selected_columns = combinations[num_combination]

    # Retorna o DataFrame filtrado e a descrição
    return df[selected_columns], description_combination

### 2.4 Definição do Alvo - Regressão

In [91]:
def define_target(df, target_column='close'):
    """
    Define a coluna de alvo para um modelo de regressão com base no preço de fechamento do próximo dia.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna que será usada como base para o cálculo do alvo. Default é 'close'.
    
    Retorna:
        pd.DataFrame: DataFrame atualizado com a nova coluna de alvo ('close_next_day') e sem a última linha.
    """
    # Trabalhar com uma cópia explícita do DataFrame para evitar avisos
    df = df.copy()
    
    # Criar a coluna de preço de fechamento do próximo dia
    df['close_next_day'] = df[target_column].shift(-1)
    
    # Remover a última linha, pois não há valor para a previsão do próximo dia
    df = df[:-1]
 
    return df, 'close_next_day'

### 2.5 Treinamento dos modelos

In [ ]:
def train_lstm_model(df, target_column, name_model):
    """
    Treina um modelo LSTM em dados de séries temporais e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).

    Retorna:
        dict: Dicionário com métricas de desempenho (Loss, MSE, RMSE, MAE, MAPE, Acurácia).
    """
    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Dividindo os dados em 80% para treino e 20% para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Escalonando os dados para que os valores estejam entre 0 e 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reshape dos dados para LSTM (adicionar dimensão para timesteps)
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    # Mapeamento de nomes de modelos para funções
    model_mapping = {
    'model_1': model_1,
    'model_2': model_2,
    'model_3': model_3,
    'model_linear_simple': model_linear_simple,
    }

    # Chamando o modelo
    if name_model == 'model_3':
        model = model_mapping[name_model]((X.shape[1],), learning_rate=0.001)
    else:
        model = model_mapping[name_model]((X_train_scaled.shape[1], X_train_scaled.shape[2]), learning_rate=0.001)
    
    # Configurando EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Treinando o modelo
    history = model.fit(
        X_train_scaled, y_train,
        #X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_test_scaled, y_test),
        callbacks=[early_stopping],
        verbose=1
    )

    # Prevendo no conjunto de teste
    y_pred = model.predict(X_test_scaled)

    # Calculando métricas adicionais
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Coeficiente de determinação (R²)
    ss_total = np.sum((y_test - np.mean(y_test)) ** 2)  # Soma dos quadrados totais
    ss_residual = np.sum((y_test - y_pred.flatten()) ** 2)  # Soma dos quadrados residuais
    r2 = 1 - (ss_residual / ss_total)

    # Acurácia considerando 2% de margem de erro
    margin = 0.02
    accuracy = accuracy_score(
        (np.abs(y_test - y_pred.flatten()) <= margin * y_test).astype(int),
        np.ones_like(y_test)
    )

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,  # Adicionando o R²
        "Acurácia": accuracy
    }

    return metrics_dict

In [143]:
def train_model_linear(df, target_column):
    """
    Treina um modelo de Regressão Linear Simples em dados de séries temporais e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).

    Retorna:
        dict: Dicionário com métricas de desempenho (Loss, MSE, RMSE, MAE, MAPE, Acurácia).
    """
    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Dividindo os dados em 80% para treino e 20% para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Escalonando os dados para que os valores estejam entre 0 e 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Chamando a função para criar o modelo de Regressão Linear Simples
    model = model_linear_simple((X_train_scaled.shape[1],))

    # Treinando o modelo
    history = model.fit(
        X_train_scaled, y_train,
        epochs=50,
        batch_size=32,
        verbose=1,
        validation_data=(X_test_scaled, y_test)
    )

    # Prevendo no conjunto de teste
    y_pred = model.predict(X_test_scaled)

    # Calculando métricas adicionais
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Coeficiente de determinação (R²)
    ss_total = np.sum((y_test - np.mean(y_test)) ** 2)  # Soma dos quadrados totais
    ss_residual = np.sum((y_test - y_pred.flatten()) ** 2)  # Soma dos quadrados residuais
    r2 = 1 - (ss_residual / ss_total)

    # Acurácia considerando 2% de margem de erro
    margin = 0.02
    accuracy = accuracy_score(
        (np.abs(y_test - y_pred.flatten()) <= margin * y_test).astype(int),
        np.ones_like(y_test)
    )

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,  # Adicionando o R²
        "Acurácia": accuracy
    }

    return metrics_dict

### 2.6 Treinamento de Todos Modelos

In [ ]:
def train_model_all(df, target_column, name_model):
    """
    Treina um modelo LSTM em dados de séries temporais e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).

    Retorna:
        dict: Dicionário com métricas de desempenho (Loss, MSE, RMSE, MAE, MAPE, Acurácia).
    """
    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Dividindo os dados em 80% para treino e 20% para teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Escalonando os dados para que os valores estejam entre 0 e 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Reshape dos dados para LSTM (adicionar dimensão para timesteps)
    X_train_scaled = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
    X_test_scaled = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

    # Mapeamento de nomes de modelos para funções
    model_mapping = {
    'model_1': model_1,
    'model_2': model_2,
    'model_3': model_3,
    'model_linear_simple': model_linear_simple,
    }

    # Chamando o modelo
    if name_model == 'model_3' or name_model == 'model_linear_simple':
        model = model_mapping[name_model]((X_train.shape[1],))
    
        # Treinando o modelo
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            verbose=1,
            validation_data=(X_test, y_test)
        )
        # Prevendo no conjunto de teste
        y_pred = model.predict(X_test)
    else:
        model = model_mapping[name_model]((X_train_scaled.shape[1], X_train_scaled.shape[2]), learning_rate=0.001)
            # Configurando EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
        # Treinando o modelo
        history = model.fit(
            X_train_scaled, y_train,
            #X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_test_scaled, y_test),
            callbacks=[early_stopping],
            verbose=1
        )
        # Prevendo no conjunto de teste
        y_pred = model.predict(X_test_scaled)
    




    # Calculando métricas adicionais
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test - y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Coeficiente de determinação (R²)
    ss_total = np.sum((y_test - np.mean(y_test)) ** 2)  # Soma dos quadrados totais
    ss_residual = np.sum((y_test - y_pred.flatten()) ** 2)  # Soma dos quadrados residuais
    r2 = 1 - (ss_residual / ss_total)

    # Acurácia considerando 2% de margem de erro
    margin = 0.02
    accuracy = accuracy_score(
        (np.abs(y_test - y_pred.flatten()) <= margin * y_test).astype(int),
        np.ones_like(y_test)
    )

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,  # Adicionando o R²
        "Acurácia": accuracy
    }

    return metrics_dict

In [138]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

def train_model_sliding_window(df, target_column, model, window_size=5, test_size=0.2, epochs=50, batch_size=32, patience=10):
    """
    Treina um modelo em dados de séries temporais usando janela deslizante e retorna as métricas de desempenho.

    Parâmetros:
        df (pd.DataFrame): DataFrame com os dados.
        target_column (str): Nome da coluna de destino (target).
        model (callable): Função que define o modelo Keras.
        window_size (int): Tamanho da janela deslizante.
        test_size (float): Proporção dos dados para teste.
        epochs (int): Número de épocas de treinamento.
        batch_size (int): Tamanho do lote para treinamento.
        patience (int): Paciência para Early Stopping.

    Retorna:
        dict: Dicionário com métricas de desempenho e histórico de treinamento.
        model: Modelo treinado.
        scaler: Scaler utilizado.
    """

    # Separando as variáveis independentes (X) e dependente (y)
    X = df.drop(columns=[target_column]).values
    y = df[target_column].values

    # Escalonando os dados
    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)

    # Criando dados com janela deslizante (usando list comprehension para concisão)
    X_sliding, y_sliding = [], []
    for i in range(len(X_scaled) - window_size):
        X_sliding.append(X_scaled[i:i + window_size])
        y_sliding.append(y[i + window_size])
    X_sliding = np.array(X_sliding)
    y_sliding = np.array(y_sliding)

    # Dividindo os dados em treino e teste (usando train_test_split para melhor controle)
    X_train, X_test, y_train, y_test = train_test_split(
        X_sliding, y_sliding, test_size=test_size, shuffle=False  # shuffle=False é crucial para séries temporais
    )

    # Instanciando o modelo
    input_shape = (X_train.shape[1], X_train.shape[2]) if len(X_train.shape) == 3 else (X_train.shape[1],)
    model = model_1(input_shape)

    # Configurando EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    # Treinando o modelo
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping],
        verbose=1
    )

    # Prevendo no conjunto de teste
    y_pred = model.predict(X_test).flatten() # Achatando o array de predições

    # Calculando métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100 if not np.any(y_test == 0) else np.inf # Tratando divisão por zero no MAPE
    r2 = r2_score(y_test, y_pred)

    # Acurácia com margem de erro (melhoria na lógica)
    margin_percentage = 0.02
    margin = margin_percentage * np.abs(y_test)
    correct_predictions = np.abs(y_test - y_pred) <= margin
    accuracy = np.mean(correct_predictions)

    # Montando o dicionário de métricas
    metrics_dict = {
        "Loss": history.history['loss'][-1],
        "Val_Loss": history.history['val_loss'][-1], # Adicionado Val_loss
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "MAPE": mape,
        "R²": r2,
        "Acurácia": accuracy,
        "history": history.history # Inclui o histórico completo para análise posterior
    }

    #return metrics_dict, model, scaler
    return metrics_dict

## 3 - Definição dos Modelos

### 3.1 Modelo 1

In [94]:
# MODEL 1

def model_1(input_shape, learning_rate=0.001):
    """
    Cria e compila um modelo LSTM.

    Parâmetros:
        input_shape (tuple): A forma da entrada (timesteps, features).
        learning_rate (float): Taxa de aprendizado para o otimizador Adam.

    Retorna:
        keras.Model: Modelo LSTM compilado.
    """
    # Criando o modelo com o uso explícito de Input
    model = Sequential([
        Input(shape=input_shape),
        LSTM(16, return_sequences=False),
        Dense(1)
    ])

    # Compilando o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mse', 'mae', 'mape']
    )

    return model

### 3.2 Modelo 2

In [101]:
# MODEL 2

def model_2(input_shape, learning_rate=0.001, dropout_rate=0.03):
    """
    Cria e compila um modelo LSTM complexo com várias camadas e dropout.

    Parâmetros:
        input_shape (tuple): A forma da entrada (timesteps, features).
        learning_rate (float): Taxa de aprendizado para o otimizador Adam.
        dropout_rate (float): Taxa de dropout aplicada nas últimas camadas.

    Retorna:
        keras.Model: Modelo LSTM compilado.
    """
    # Criando o modelo com o uso explícito de Input
    model = Sequential([
        Input(shape=input_shape),
        LSTM(200, activation='tanh', recurrent_activation='sigmoid', return_sequences=True),
        LSTM(300, activation='tanh', recurrent_activation='sigmoid', return_sequences=True),
        LSTM(400, activation='tanh', recurrent_activation='sigmoid', return_sequences=True),
        Dropout(dropout_rate),  # Dropout na penúltima camada LSTM
        LSTM(400, activation='tanh', recurrent_activation='sigmoid', return_sequences=False),
        Dropout(dropout_rate),  # Dropout na última camada LSTM
        Dense(1)  # Camada de saída
    ])

    # Compilando o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mse', 'mae', 'mape']
    )

    return model

### 3.3 Modelo 3

In [123]:
# Modelo 3
def model_3(input_shape):
    """
    Cria e compila um modelo denso com 3 camadas.

    Parâmetros:
        input_shape (tuple): A forma da entrada (features,).
        learning_rate (float): Taxa de aprendizado para o otimizador Adam.

    Retorna:
        keras.Model: Modelo denso compilado.
    """
    learning_rate=0.001
    
    # Criando o modelo
    model = Sequential([
        Input(shape=input_shape),
        Dense(32, activation='relu'),  # Primeira camada Dense
        Dense(8, activation='relu'),                            # Segunda camada Dense
        Dense(1)                                                # Camada de saída
    ])

    # Compilando o modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mse', 'mae', 'mape']
    )

    return model

### 3.4 Modelo 4 - model_linear_simple

In [121]:
# MODEL: Regressão Linear Simples

def model_linear_simple(input_shape):
    """
    Cria um modelo de Regressão Linear Simples.

    Parâmetros:
        input_shape (tuple): A forma da entrada (features,).

    Retorna:
        keras.Model: Modelo de regressão linear simples compilado.
    """
    from keras.models import Sequential
    from keras.layers import Dense

    # Criando o modelo
    model = Sequential([
        Input(shape=input_shape),
        Dense(1, activation='linear')  # Saída linear para regressão
    ])

    # Compilando o modelo
    model.compile(
        optimizer='adam',  # Otimizador Adam padrão
        loss='mean_squared_error',  # Perda de erro médio quadrático
        metrics=['mse', 'mae', 'mape']  # Métricas úteis para regressão
    )

    return model

## 4 - Treino dos modelos

### 4.1 Função Principal - process_stocks_and_save_metrics

In [144]:
def process_stocks_and_save_metrics(stock_list, num_combination, name_model):
    """
    Processa uma lista de ações, treina o modelo LSTM para cada uma e salva as métricas em um DataFrame.

    Parâmetros:
        stock_list (list): Lista de ações (str).
        get_stock_data_func (function): Função para obter os dados de cada ação.
            Deve retornar um DataFrame com os dados da ação e o nome da coluna de destino (target).

    Retorna:
        pd.DataFrame: DataFrame contendo o nome da ação, descrição dos dados, nome do modelo e as métricas.
    """
    results = []

    for stock in stock_list:
        print(f"{name_model}: Processando ação: {stock}, combination: {num_combination}...")

        # Obtém os dados da ação
        df = all_stock_data[stock]
        # Realizar as combination
        df, description_combination = make_combination_data(df, num_combination)
        # Definir o alvo
        df, target_column = define_target(df)

        # Treina o modelo e coleta as métricas
        #metrics = train_lstm_model(df, target_column, name_model)
        #metrics = train_model_all(df, target_column, name_model)
        #metrics = train_model_windows(df, target_column, name_model)
        #metrics = train_model_linear(df, target_column)
        metrics = train_model_sliding_window(df, target_column, name_model) 

        # Salva os resultados no formato desejado
        results.append({
            "Stock": stock,
            "Description": description_combination,
            "Model": name_model,
            **metrics
        })

    # Retorna os resultados em um DataFrame
    return pd.DataFrame(results)

    # Normalização
    # Normalizar os colunas
    #columns = ['close','open', 'high', 'low', 'mms_5', 'mms_30', 'mme_5','mme_30']
    
    # Normalizar
    #df = normalize_columns_2(df, columns, 30)
    
    return df, 'close_next_day', description_combination, 

### 4.2 Rodando em todas as Empresas

In [145]:
# Processando as ações com barra de progresso
'''
Opções de modelos
1: model_1
2: model_2
3: model_3
4: model_linear_simple

'''

# Lista de ações
#stock_list = ['PETR4', 'VALE3', 'BBDC4','ITUB4']
stock_list = ['PETR4']

# Carregar os dados
all_stock_data = load_stocks_data(stock_list)

name_model = 'model_1'
results_list = []

# Usando tqdm para exibir progresso
for i in tqdm(range(7), desc="Processando combinações"):
    results_list.append(process_stocks_and_save_metrics(stock_list, i + 1, name_model))
    #results_list.append(process_stocks_and_save_metrics(stocks, get_stock_data, i + 1, name_model))

# Combina os resultados em um único DataFrame
results_df = pd.concat(results_list, ignore_index=True)

# Exibir o resultado
results_df
# Salvar em CSV
results_df.to_csv(f'Estudos_IEEE_{name_model}.csv', sep=';', encoding='utf-8-sig')

Carregando dados para: PETR4


Processando combinações:   0%|                                                                   | 0/7 [00:00<?, ?it/s]

model_1: Processando ação: PETR4, combination: 1...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.1252 - mae: 0.2699 - mape: 38263.0078 - mse: 0.1252 - val_loss: 0.0019 - val_mae: 0.0330 - val_mape: 8.4687 - val_mse: 0.0019
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0032 - mae: 0.0440 - mape: 104138.7031 - mse: 0.0032 - val_loss: 9.3976e-04 - val_mae: 0.0212 - val_mape: 5.1139 - val_mse: 9.3976e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.8550e-04 - mae: 0.0215 - mape: 2402.4290 - mse: 7.8550e-04 - val_loss: 9.0899e-04 - val_mae: 0.0211 - val_mape: 5.0572 - val_mse: 9.0899e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.3399e-04 - mae: 0.0185 - mape: 25120.5117 - mse: 6.3399e-04 - val_loss: 9.0423e-04 - val_mae: 0.0211 - val_mape: 5.0573 - val_mse: 9.0423e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5302e-04 - mae: 0.0176 - mape: 14238.7979 - mse: 5.5302e-04 - val_loss: 8.9438e-04 - val_mae: 0.0210 - v

Processando combinações:  14%|████████▍                                                  | 1/7 [00:16<01:37, 16.22s/it]

model_1: Processando ação: PETR4, combination: 2...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.0749 - mae: 0.2009 - mape: 117237.2266 - mse: 0.0749 - val_loss: 0.0060 - val_mae: 0.0657 - val_mape: 13.7765 - val_mse: 0.0060
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0029 - mae: 0.0423 - mape: 234928.7344 - mse: 0.0029 - val_loss: 0.0010 - val_mae: 0.0237 - val_mape: 5.6118 - val_mse: 0.0010
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.2172e-04 - mae: 0.0222 - mape: 7246.3711 - mse: 8.2172e-04 - val_loss: 9.0723e-04 - val_mae: 0.0218 - val_mape: 5.2350 - val_mse: 9.0723e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.6296e-04 - mae: 0.0176 - mape: 4033.7913 - mse: 5.6296e-04 - val_loss: 9.5845e-04 - val_mae: 0.0220 - val_mape: 5.2542 - val_mse: 9.5845e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.1979e-04 - mae: 0.0167 - mape: 3418.2686 - mse: 5.1979e-04 - val_loss: 9.3511e-04 - val_mae: 0.0224 - val_mape:

...

model_1: Processando ação: PETR4, combination: 6...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.1347 - mae: 0.2849 - mape: 12099.3730 - mse: 0.1347 - val_loss: 0.0063 - val_mae: 0.0651 - val_mape: 14.4124 - val_mse: 0.0063
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0062 - mae: 0.0610 - mape: 5440.0195 - mse: 0.0062 - val_loss: 0.0026 - val_mae: 0.0390 - val_mape: 9.4216 - val_mse: 0.0026
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0015 - mae: 0.0299 - mape: 1697.2064 - mse: 0.0015 - val_loss: 0.0014 - val_mae: 0.0260 - val_mape: 6.4484 - val_mse: 0.0014
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.8177e-04 - mae: 0.0216 - mape: 20153.7637 - mse: 7.8177e-04 - val_loss: 0.0012 - val_mae: 0.0234 - val_mape: 5.7926 - val_mse: 0.0012
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.3595e-04 - mae: 0.0190 - mape: 55989.8906 - mse: 6.3595e-04 - val_loss: 0.0012 - val_mae: 0.0225 - val_mape: 5.5801 - val_mse: 0.0012
Epo

Processando combinações:  86%|██████████████████████████████████████████████████▌        | 6/7 [01:39<00:16, 16.65s/it]

model_1: Processando ação: PETR4, combination: 7...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0711 - mae: 0.2000 - mape: 169890.4375 - mse: 0.0711 - val_loss: 0.0047 - val_mae: 0.0562 - val_mape: 12.6740 - val_mse: 0.0047
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0040 - mae: 0.0478 - mape: 25047.4707 - mse: 0.0040 - val_loss: 0.0016 - val_mae: 0.0290 - val_mape: 7.2537 - val_mse: 0.0016
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0010 - mae: 0.0246 - mape: 57138.6523 - mse: 0.0010 - val_loss: 0.0011 - val_mae: 0.0234 - val_mape: 5.8852 - val_mse: 0.0011
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.0470e-04 - mae: 0.0195 - mape: 2794.4709 - mse: 7.0470e-04 - val_loss: 0.0011 - val_mae: 0.0233 - val_mape: 5.7322 - val_mse: 0.0011
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.9151e-04 - mae: 0.0190 - mape: 19794.3574 - mse: 6.9151e-04 - val_loss: 0.0010 - val_mae: 0.0222 - val_mape: 5.4905 - val_mse: 0.0010
E

Processando combinações: 100%|███████████████████████████████████████████████████████████| 7/7 [01:56<00:00, 16.60s/it]


,Stock,Description,Model,Loss,Val_Loss,MSE,RMSE,MAE,MAPE,R²,Acurácia,history
0,PETR4,Stock Data,model_1,0.000352,0.000777,0.000636,0.025225,0.017014,4.061247,0.937021,0.384401,"{'loss': [0.0537283830344677, 0.00221196212805..."
1,PETR4,Stock Data + Google News,model_1,0.000336,0.000667,0.000623,0.024964,0.017345,4.100763,0.938319,0.344011,"{'loss': [0.03396504744887352, 0.0020460460800..."
2,PETR4,Stock Data + Twitter,model_1,0.000355,0.000649,0.000608,0.024658,0.016720,3.970200,0.939821,0.401114,"{'loss': [0.022044915705919266, 0.001054765889..."
3,PETR4,Stock Data + IT (IFR + MMS + MME),model_1,0.000346,0.000596,0.000574,0.023961,0.016339,3.884865,0.943177,0.410864,"{'loss': [0.017256632447242737, 0.000663439219..."
4,PETR4,Google News + Twitter + IFR + MMS,model_1,0.000354,0.000660,0.000655,0.025599,0.017332,4.203560,0.935141,0.385794,"{'loss': [0.03331499546766281, 0.0052486308850..."
5,PETR4,Google News + Twitter + IFR + MME,model_1,0.000368,0.000730,0.000714,0.026716,0.018081,4.417148,0.929360,0.369081,"{'loss': [0.05916878581047058, 0.0041926926933..."
6,PETR4,Google News + Twitter + IFR + MME + MMS,model_1,0.000355,0.000649,0.000649,0.025468,0.017931,4.298642,0.935802,0.350975,"{'loss': [0.034955598413944244, 0.002614361234..."
